In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
import sklearn
import matplotlib.pyplot as plt
import os
import sys
X_train, X_test, y_train, y_test = np.load('D:/article/npy/clahe.npy', allow_pickle = True)

# Define the TensorFlow model
model_path = 'D:/article/result/Dense_clahe_20/model.h5'
size_bytes = os.path.getsize(model_path)
model = load_model(model_path)
model.summary()
size_mb = size_bytes / (1024 * 1024)
print(f"Model size: {size_mb:.2f} MB")
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
confusion_mtx =  sklearn.metrics.confusion_matrix(y_test_classes, y_pred_classes, labels=None, sample_weight=None, normalize=None)
plt.imshow(confusion_mtx, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(4)
plt.xticks(tick_marks, ['first', 'second','third','out of'])
plt.yticks(tick_marks, ['first', 'second','third','out of'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='macro')
recall = recall_score(y_test_classes, y_pred_classes, average='macro')
f1 = f1_score(y_test_classes, y_pred_classes, average='macro')
print('Accuracy: {:.4f}'.format(accuracy))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 score: {:.4f}'.format(f1))
with open("D:/article/result/clahe_cnn_20/performance.txt", "w") as f:
    f.write(f"Accuracy: {round(accuracy, 4)}\nPrecision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1_score: {round(f1, 4)}")

In [ ]:
y_test_cat = to_categorical(y_test_classes, num_classes=4)
y_pred_cat = to_categorical(y_pred_classes, num_classes=4)
# Calculate ROC AUC score
auc = roc_auc_score(y_test_cat, y_pred_cat, multi_class='ovo')

# Calculate FPR and TPR values for ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_test_cat[:, i], y_pred_cat[:, i])
    roc_auc[i] = roc_auc_score(y_test_cat[:, i], y_pred_cat[:, i])
# Plot the ROC curve
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'red']
for i, color in zip(range(4), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (AUC = {1:0.2f})'
             ''.format(i+1, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()
print('AUC: {:.4f}'.format(auc))
